# Test: Bestimmung Precision / Recall mit SVM
Um die Klassifikation, und damit auch von dem Aufbau unserer Matrix, zu testen, bestimmen wir Precision und Recall mit einer SVM.

* Precision: Wieviele der zugeordneten Samples sind tatsächlich relevant? (true pos.) / (true pos. + false pos.)
* Recall: Wieviele der tatsächlich relevanten Samples wurden gefunden / zugeordnet? (true pos.) / (true pos. + false neg.)
 * Tiefe Precision / hoher Recall: Es werden viele Samples zugeordnet, von denen jedoch viele falsch sind
 * Hohe Precision / tiefer Recall: Es werden nur wenige Samples zugeordnet, von denen jedoch die meisten stimmen.
<img src="img/Precisionrecall.png" alt="Drawing" style="width: 200px"/>

## Optimierung
Mit diesem Test versuchen wir die Konfiguration der SVM möglichst gut auf unsere Datenstruktur auszurichten.
### Kernel
Der Kernel definiert, mit welcher Art Funktion die Datenpunkte im 2D-Raum voneinander getrennt werden. Standardmässig wird dazu eine lineare Funktion (linear kernel) verwendet. Wenn die Punkte im training set nicht durch eine Gerade trennbar sind, versucht man sie in einer höheren Dimension voneinander zu trennen. Die erreicht man durch andere kernel-Funktionen (z.B. Polynomial oder RBF).
Bei vielen Features, wie es bei unseren Daten der Fall ist, bringt eine höhere Dimension keine Vorteile, ist aber sehr teuer. Entsprechend verwenden wir einen linearen Kernel.

#### Parameter
Der lineare Kernel verlangt ausschliesslich einen Penalty Parameter C. Beim einem hohen C versucht der Classifier die Ebene so zu legen, dass möglichst wenige Datenpunkte aus dem training set falsch klassifiziert sind. Dafür nimmt er ein kleineres Margin zwischen Ebene und Punkte in Kauf. Bei einem tiefen Wert für C sucht der Classifier nach einem möglichst hohen margin, nimmt aber falsche Klassifizierungen im trianing set in Kauf.

### Bestimmung Parameter C
Evaluation mit 2000 samples im test set:
* C=0.1: Precision 0.777777777778, Recall 0.21875
* C=0.2: Precision 0.785714285714, Recall 0.34375
* C=0.3: Precision 0.769230769231, Recall 0.3125
* C=0.4: Precision 0.714285714286, Recall 0.3125
* C=0.5: Precision 0.666666666667, Recall 0.3125
* C=0.6: Precision 0.555555555556, Recall 0.3125
* C=0.7: Precision 0.555555555556, Recall 0.3125
* C=0.8: Precision 0.578947368421, Recall 0.34375
* C=0.9: Precision 0.578947368421, Recall 0.34375
* C=1: Precision 0.578947368421, Recall 0.34375

### Test mit Parameter C
#### Test mit Matrix V1
C=0.2
* Samples 1000: Precision 1.0, Recall 0.240740740741
* Samples 2000: Precision 0.833333333333, Recall 0.37037037037
* Samples 5000: Precision 0.785714285714, Recall 0.407407407407
* Samples 10000: Precision 0.821428571429, Recall 0.425925925926
* Samples 15000: Precision 0.896551724138, Recall 0.481481481481

#### Test mit Matrix V2 
C=0.2
* Samples 1000: Precision 1.0, Recall 0.327272727273
* Samples 2000: Precision 0.9375, Recall 0.545454545455
* Samples 5000: Precision 0.885714285714, Recall 0.563636363636
* Samples 10000: Precision 0.861111111111, Recall 0.563636363636
* Samples 15000: Precision 0.916666666667, Recall 0.6

Samples=15000
* C=0.2: Precision 0.916666666667, Recall 0.6
* C=0.6: Precision 0.888888888889, Recall 0.727272727273
* C=1.0: Precision 0.816326530612, Recall 0.727272727273

#### Test mit 1/3 und 2/3 bei 15000 Samples
* #1: Precision 0.955922865014, Recall 0.805104408353
* #2: Precision 0.937555753791, Recall 0.812838360402
* #3: Precision 0.956014362657, Recall 0.823665893271
* #4: Precision 0.950628366248, Recall 0.819025522042
* #5: Precision 0.946654611212, Recall 0.809744779582

#### Test mit Matrix V3
* #1: Precision 0.89219858156, Recall 0.643807574207

#### Test mit Klassifikationsmatrix
* #1: Precision 0.89248434238, Recall 0.707196029777
* #2: Precision 0.887804878049, Recall 0.752688172043
* #3: Precision 0.897177419355, Recall 0.736145574855
* #4: Precision 0.881592039801, Recall 0.732837055418
* #5: Precision 0.893551688843, Recall 0.722084367246
* #6: Precision 0.892966360856, Recall 0.724565756824
* #7: Precision 0.89296333003, Recall 0.745244003309
* #8: Precision 0.895895895896, Recall 0.740281224152
* #9: Precision 0.886294416244, Recall 0.722084367246
* #10: Precision 0.896586345382, Recall 0.738626964433
* #11: Precision 0.8921859545, Recall 0.746071133168
* #12: Precision 0.908163265306, Recall 0.736145574855
* #13: Precision 0.880445795339, Recall 0.718775847808
* #14: Precision 0.890816326531, Recall 0.722084367246
* #15: Precision 0.907237512742, Recall 0.736145574855
* 
* Schnitt Precision: 0.89307770475
* Schnitt Recall: 0.732065067549

#### Test mit Klassifikationsmatrix nach dem Bugfix
* #1: Precision 0.806193806194, Recall 0.66694214876

In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np

from imports.matrix_helper import MatrixHelper
from sklearn import svm
from sklearn.metrics import precision_recall_curve

matrix_helper = MatrixHelper()

# Read pickle
feature_matrix = matrix_helper.load_from_parse('data/feature_matrix_sparse_classification.pickle')[0]

feature_matrix = feature_matrix[:1000, :]

# Create own matrices for vulenrable and not vulnerable components
vulnerable, not_vulnerable = matrix_helper.split_vulnerable_notvulnerable(feature_matrix)

# Variables for Iteration
precision_sum = 0
recall_sum = 0
number_of_iterations = 5


for i in range(number_of_iterations):
    # Split into training sets (2/3) and test sets (1/3)
    not_vulnerable_training, not_vulnerable_test = matrix_helper.split_training_test(not_vulnerable, 2.0/3)
    vulnerable_training, vulnerable_test = matrix_helper.split_training_test(vulnerable, 2.0/3)

    # Concatenate vulnerable/not-vulnerable and split into training and target matrices
    training_data, training_target = matrix_helper.create_data_target(not_vulnerable_training, vulnerable_training)
    test_data, test_target = matrix_helper.create_data_target(not_vulnerable_test, vulnerable_test)

    # Create classifier
    clf = svm.SVC(kernel='linear', C=0.2)

    # Fit model
    clf.fit(training_data, training_target)

    # Predict remaining data
    target_prediction = clf.predict(test_data)

    # Compute Precision-Recall
    precision, recall, thresholds = precision_recall_curve(test_target, target_prediction)

    precision_sum += precision[1]
    recall_sum += recall[1]

    # Print Precision and Recall
    print('* #{} Precision {}, Recall {}'.format(i, precision[1], recall[1]))

print('Precision mean: {}'.format(precision_sum / number_of_iterations))
print('Recall mean: {}'.format(recall_sum / number_of_iterations))




* #0 Precision 0.673469387755, Recall 0.568965517241
* #1 Precision 0.706896551724, Recall 0.706896551724
* #2 Precision 0.708333333333, Recall 0.586206896552
* #3 Precision 0.76, Recall 0.655172413793
* #4 Precision 0.62962962963, Recall 0.586206896552
Precision mean: 0.695665780488
Recall mean: 0.620689655172


In [3]:
import time
import matplotlib.pyplot as plt
import numpy as np

from imports.matrix_helper import MatrixHelper
from sklearn import svm
from sklearn.metrics import precision_recall_curve

matrix_helper = MatrixHelper()

# Read pickle
feature_matrix = matrix_helper.load_from_parse('data/feature_matrix_sparse_classification.pickle')[0]

# Create own matrices for vulenrable and not vulnerable components
vulnerable, not_vulnerable = matrix_helper.split_vulnerable_notvulnerable(feature_matrix)

# Split into training sets (2/3) and test sets (1/3)
not_vulnerable_training, not_vulnerable_test = matrix_helper.split_training_test(not_vulnerable, 2.0/3)
vulnerable_training, vulnerable_test = matrix_helper.split_training_test(vulnerable, 2.0/3)

# Concatenate vulnerable/not-vulnerable and split into training and target matrices
training_data, training_target = matrix_helper.create_data_target(not_vulnerable_training, vulnerable_training)
test_data, test_target = matrix_helper.create_data_target(not_vulnerable_test, vulnerable_test)

start = time.time()

# Create classifier
clf = svm.SVC(kernel='linear', C=0.2)

# Fit model
clf.fit(training_data, training_target)

# Predict data
target_prediction = clf.predict(test_data)

# Compute Precision-Recall
precision, recall, thresholds = precision_recall_curve(test_target, target_prediction)

end = time.time()
elapsed = (end - start) / 60

# Print
print("Verwundbare Komponenten im Testset: {0:10.1f}%".format(matrix_helper.get_vulnerable_percentage(test_target)))
print("Verwundbare Komponenten in Vorhersage: {0:10.1f}%".format(matrix_helper.get_vulnerable_percentage(target_prediction)))
print('------------------------------')
print('Precision {}, Recall {}'.format(precision[1], recall[1]))
print('time: {0:10.1f}min'.format(elapsed))

Verwundbare Komponenten im Testset:       18.6%
Verwundbare Komponenten in Vorhersage:       15.4%
------------------------------
Precision 0.806193806194, Recall 0.66694214876
time:       23.5min
